In [7]:
import gymnasium as gym
import numpy as np

In [2]:
env = gym.make("CartPole-v1", render_mode = "human")

In [43]:
max_obs_values = env.observation_space.high
min_obs_values = env.observation_space.low
print(f"Actual observation max values: {max_obs_values}")
print(f"Actual observation min values: {min_obs_values}")
# This is kinda a lot, and we dont need infinity values, we can change/ignore them
# and we can change radians to degree too

# lets change them, not sure about this yet
# max_obs_values = (max_obs_values[0], 20, np.rad2deg(max_obs_values[2]), 20)
# min_obs_values = (min_obs_values[0], -20, np.rad2deg(min_obs_values[2]), -20)

# lets ignore them, simpler observations
# only position and angle is used
max_obs_values = np.array([max_obs_values[0], np.rad2deg(max_obs_values[2])])
min_obs_values = np.array([min_obs_values[0], np.rad2deg(min_obs_values[2])])
print(f"Used observation max values: {max_obs_values}")
print(f"Used observation min values: {min_obs_values}")

Actual observation max values: [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
Actual observation min values: [-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]
Used observation max values: [ 4.8 24. ]
Used observation min values: [ -4.8 -24. ]


In [66]:
DISCRETE_OBS_SPACE_SIZE = [20]* len(max_obs_values) 
# DISCRETE_OBS_SPACE_SIZE = [20, 30] # these are the what the max values will corresponds to
discrete_obs_space_step_size = (max_obs_values - min_obs_values) / DISCRETE_OBS_SPACE_SIZE
discrete_obs_space_step_size

array([0.48000002, 1.6       ])

In [99]:
def discretizer(obs):
    obs = np.array([obs[0], obs[2]])
    discrete_obs = (obs - min_obs_values)/discrete_obs_space_step_size
    return tuple(discrete_obs.astype(np.int16))
disc_obs = discretizer([-4.8, 2, 22, 6]) 
disc_obs
# min value corresponds to zero
# max value corresponds to discrete space size for that index

(0, 28)

In [85]:
action_space = env.action_space.n
action_space
(DISCRETE_OBS_SPACE_SIZE + [env.action_space.n])

[20, 30, 2]

In [100]:
q_table = np.random.uniform(low=-2,high=2, size=(DISCRETE_OBS_SPACE_SIZE + [env.action_space.n]))
q_table[disc_obs]

array([0.6998863 , 1.09658777])